<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [57]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import os
import requests
from bs4 import BeautifulSoup as bs

load_dotenv()

True

In [58]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = os.getenv('DBNAME')
USER = os.getenv('S_USER')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

In [65]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [60]:
# текст запроса
query_3_1 = f'''SELECT COUNT(id)
                FROM public.vacancies'''

In [66]:
# результат запроса
def get_count(base_request):
    result = None
    try:
        with connection.cursor() as cursor:
            cursor.execute(base_request)
            result = cursor.fetchone()
    except Exception as e:
        print(e)
    finally:
        cursor.close()
    return result[0] if result else False

print(get_count(query_3_1))

49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [ ]:
# текст запроса
request_3_2 = f'''SELECT COUNT(id)
                FROM public.employers'''

In [ ]:
# результат запроса
print(get_count(request_3_2))

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [ ]:
# текст запроса
request_3_3 = f'''SELECT COUNT(id)
                FROM public.areas'''

In [ ]:
# результат запроса
print(get_count(request_3_3))

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [ ]:
# текст запроса
request_3_4 = f'''SELECT COUNT(id)
                FROM public.industries'''

In [ ]:
# результат запроса
print(get_count(request_3_4))

***

In [ ]:
# выводы по предварительному анализу данных

В базе имеется около 50 тысяч различных вакансий от 23501 работодателя, т.е. в среднем чуть больше 2 вакансий на одного работодателя. Работодатели рассредоточены по 1362 регионам и в среднем в каждом регионе около 17 работодателей и около 35 вакансий. На каждую сферу деятельности в среднем приходится около 80 работодателей и около 160 вакансий.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [ ]:
# текст запроса
request_4_1 = f'''SELECT a.name, COUNT(v.id)
                FROM public.vacancies as v
                JOIN public.areas as a ON v.area_id = a.id
                GROUP BY a.name
                ORDER BY 2 DESC
                LIMIT 5'''

In [ ]:
# результат запроса
df = pd.read_sql_query(request_4_1, connection)
df


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [ ]:
# текст запроса
request_4_2 = f'''SELECT COUNT(*)
                FROM public.vacancies as v
                WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL'''

In [ ]:
# результат запроса
print(get_count(request_4_2))

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [ ]:
# текст запроса
request_4_3 = f'''SELECT ROUND(AVG(salary_from)) as from, ROUND(AVG(salary_to)) as to
                FROM public.vacancies
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(request_4_3, connection)
df

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [ ]:
# текст запроса
request_4_4 = f'''SELECT schedule, employment, COUNT(id)
                FROM public.vacancies
                GROUP BY schedule, employment
                ORDER BY 3 DESC
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(request_4_4, connection)
df

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [ ]:
# текст запроса
request_4_5 = f'''SELECT experience, COUNT(id)
                FROM public.vacancies
                GROUP BY experience
                ORDER BY 2
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(request_4_5, connection)
df

***

In [ ]:
# выводы по детальному анализу вакансий


Наибольшее числов вакансий наблюдается в наиболее крупных промышленных центрах бывшего СССР рейтинг возглавляют Москва и Санкт-Петербург. В первую пятерку лидеров входят также столица Беларусии.
Около половины вакансий содержат сведения о предлагаемой оплате труда, что может означать нежелание разглашать свою меньшую конкурентноспособность работодателей на рынке труда.
Средняя начальная и средняя максимальная оплата труда составляют вилку от 71 до 110 с лишним тысяч рублей, что в одних регионах может быть приемлемой оплатой труда, а в других совершенно недостаточной. Начальная и максимальная оплата труда отличаются более чем в полтора раза, что в принципе определяет потенциал роста соискателей в плане повышения квалификации.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
# текст запроса
request_5_1 = f'''(SELECT e.name, COUNT(v.id)
                FROM public.vacancies as v
                JOIN public.employers as e ON v.employer_id=e.id
                GROUP BY e.name
                ORDER BY 2 DESC
                LIMIT 1)
                UNION
                (SELECT e.name, COUNT(v.id)
                FROM public.vacancies as v
                JOIN public.employers as e ON v.employer_id=e.id
                GROUP BY e.name
                ORDER BY 2 DESC
                LIMIT 1 OFFSET 4)
                ORder BY 2 DESC
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(request_5_1, connection)
df

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# текст запроса
request_5_2 = f'''SELECT
                    a.name areas_name,
                    COUNT(e.id) count_employers,
                    COUNT(v.id) count_vacancies
                FROM 
                    public.areas a 
                    LEFT JOIN public.employers e ON a.id = e.area
                    LEFT JOIN public.vacancies v ON a.id = v.area_id
                GROUP BY a.id
                ORDER BY 3, 2 DESC
                LIMIT 1
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(request_5_2, connection)
df

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# текст запроса
request_5_3 = f'''SELECT 
                    e.name employer_name,
                    COUNT(DISTINCT a.id) count_areas
                FROM 
                    public.employers e 
                    FULL OUTER JOIN public.vacancies v ON e.id = v.employer_id
                    LEFT JOIN public.areas a ON a.id = v.area_id
                GROUP BY e.id
                ORDER BY 2 DESC
                LIMIT 1
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(request_5_3, connection)
df

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
# текст запроса
request_5_4 = f'''SELECT 
                    COUNT(e.id)
                FROM public.employers e 
                LEFT JOIN employers_industries ei ON e.id=ei.employer_id
                WHERE ei.employer_id is NULL
                '''

In [ ]:
# результат запроса
print(get_count(request_5_4))

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
# текст запроса
request_5_5 = f'''SELECT 
                    e.name
                FROM public.employers e 
                FULL JOIN employers_industries ei ON e.id=ei.employer_id
                GROUP BY e.name
                HAVING COUNT(ei.industry_id) = 4
                ORDER BY 1
                OFFSET 2
                LIMIT 1                
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(request_5_5, connection)
df

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
# текст запроса
request_5_6 = f'''SELECT
                    COUNT(e.id)
                FROM public.employers e 
                FULL JOIN employers_industries ei ON e.id=ei.employer_id
                WHERE ei.industry_id = (
                    SELECT id
                    FROM public.industries
                    WHERE name ilike 'Разработка программного обеспечения'
                )          
                '''

In [ ]:
# результат запроса
print(get_count(request_5_6))

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
# код для получения списка городов-милионников
def get_cities(table) -> list:
    """Возвращает словарь вида {Город: Население(тыс.чел)}

    Args:
        bs4.element: "элемент BeautifulSoup тэга tbody"

    Returns:
        list: словарь вида {Город: Население(тыс.чел)}
    """
    cities_dict = {}
    for line in table[1:]:
        cells = line.find_all('td')
        cities_dict[cells[1].string] = int(cells[2].string.replace(' ', ''))
    return cities_dict
    
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров'
response = requests.get(url)
page = bs(response.text, 'html.parser')
tables = page.find_all('tbody')
millions_table = tables[0].find_all('tr')
cities = tuple(get_cities(millions_table).keys())
print(cities)

In [ ]:
# текст запроса
request_5_7 = f'''with ts as (SELECT
                    a.name as name,
                    COUNT(e.id) as count
                FROM public.vacancies v 
                JOIN public.areas a ON v.area_id=a.id
                JOIN public.employers e ON v.employer_id=e.id
                WHERE e.name='Яндекс'
                GROUP BY a.name
                HAVING a.name in {cities}
                ORDER BY 2)
                
                SELECT ts.name, ts.count
                FROM ts
                UNION
                SELECT 'total', SUM(ts.count)
                FROM ts
                ORDER BY 2      
                '''

In [ ]:
# результат запроса
df = pd.read_sql_query(request_5_7, connection)
df

***

In [ ]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса
request_6_1 = f'''SELECT
                    COUNT(v.id) as count
                FROM public.vacancies v 
                WHERE name ilike '%data%' OR name ilike '%данн%'
                '''

In [ ]:
# результат запроса
print(get_count(request_6_1))

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса
request_6_2 = f'''SELECT
                    COUNT(v.id) as count
                FROM public.vacancies v 
                WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                AND (name ILIKE '%junior%'
                OR experience='Нет опыта'
                OR employment='Стажировка')
                '''

In [ ]:
# результат запроса
print(get_count(request_6_2))

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса
request_6_3 = f'''SELECT
                    COUNT(v.id) as count
                FROM public.vacancies v 
                WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name ILIKE '%ML%' AND name NOT LIKE '%HTML%' AND name NOT LIKE '%QML%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                AND (key_skills LIKE '%SQL%' OR key_skills LIKE '%postgres%')
                '''

In [ ]:
# результат запроса
print(get_count(request_6_3))

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса
request_6_4 = f'''SELECT
                    COUNT(v.id) as count
                FROM public.vacancies v 
                WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name ILIKE '%ML%' AND name NOT LIKE '%HTML%' AND name NOT LIKE '%QML%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                AND key_skills ILIKE '%python%'
                '''

In [ ]:
# результат запроса
print(get_count(request_6_4))

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [84]:
# текст запроса
request_6_5 = f'''SELECT
                    ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9),'')) + 1), 2)
                FROM public.vacancies v 
                WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                '''

In [85]:
# результат запроса
df = pd.read_sql_query(request_6_5, connection)
df

/tmp/ipykernel_43941/4157864644.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(request_6_5, connection)


,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [90]:
# текст запроса
request_6_5 = f'''SELECT
                    DISTINCT experience,
                    ROUND(AVG(coalesce((v.salary_from+v.salary_to)/2, v.salary_from, v.salary_to)))
                FROM public.vacancies v 
                WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                AND (v.salary_from IS NOT Null or v.salary_to Is NOT Null)
                GROUP BY 1
                '''

In [91]:
# результат запроса
df = pd.read_sql_query(request_6_5, connection)
df

/tmp/ipykernel_43941/402798331.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(request_6_5, connection)


,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

In [ ]:
# выводы по предметному анализу

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования